## Cross in Tray 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/globtim`


In [2]:
# Constants and Parameters
const n, a, b = 2, 10, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 5e-2            # Define the tolerance for the L2-norm
f = CrossInTray # Objective function

CrossInTray (generic function with 1 method)

### Questions to fix:
 - [ ]: Why are we increasing the degree of the approximant and why are we using such a large number of points? The constructor only specifies a few. 
 - [x]: Filter critical points.

In [3]:
d = 8 # Initial Degree 
SMPL = 10 # Number of samples
center = [0.0, 0.0]
TR = test_input(f,
    dim=n,
    center=[0.0, 0.0],
    GN=SMPL,
    sample_range=scale_factor, 
    tolerance=tol_l2,
)
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

current L2-norm: 6.086325213629537
current L2-norm: 8.98641303149505


In [4]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [5]:
real_pts_cheb = solve_polynomial_system(x, TR.dim, pol_cheb.degree, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, TR.dim, pol_lege.degree, pol_lege.coeffs; basis=:legendre, bigint=true)

Tracking 49 paths...   4%|█▎                            |  ETA: 0:01:10



Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  49
  # non-singular solutions (real):  49 (49)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (49)
Tracking 49 paths...   4%|█▎                            |  ETA: 0:00:30



Tracking 49 paths...  47%|██████████████▏               |  ETA: 0:00:02



Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:01
  # paths tracked:                  49
  # non-singular solutions (real):  49 (49)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (49)


49-element Vector{Vector{Float64}}:
 [0.9262920181887633, 0.9262920181887633]
 [0.6894554893184359, 0.9254010019393569]
 [-9.89628114247498e-18, 0.9269480323779509]
 [-0.6894554893184363, 0.9254010019393569]
 [-0.9262920181887637, 0.9262920181887632]
 [-0.40672481030371405, 0.9262111529364568]
 [0.40672481030371405, 0.9262111529364567]
 [0.9254010019393573, 0.6894554893184358]
 [0.688498628425798, 0.6884986284257977]
 [-2.0321558641462446e-17, 0.6912803233191668]
 ⋮
 [-0.40207252728011716, -0.402072527280117]
 [0.40207252728011716, -0.402072527280117]
 [0.926211152936457, 0.40672481030371427]
 [0.6894409038205237, 0.407769362336129]
 [-2.891067539909121e-17, 0.39615578635009774]
 [-0.6894409038205237, 0.407769362336129]
 [-0.926211152936457, 0.40672481030371416]
 [-0.4020725272801171, 0.4020725272801169]
 [0.4020725272801171, 0.4020725272801169]

In [6]:
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, TR)
df_lege = process_critical_points(real_pts_lege, f, TR)


=== Starting Critical Points Processing (dimension: 2) ===
Processed 49 points (0.035s)

=== Starting Critical Points Processing (dimension: 2) ===
Processed 49 points (0.0s)


Row,x1,x2,z
,Float64,Float64,Float64
1,9.26292,9.26292,-10.0763
2,6.89455,9.25401,-12.0875
3,-9.89628e-17,9.26948,-0.341423
4,-6.89455,9.25401,-12.0875
5,-9.26292,9.26292,-10.0763
6,-4.06725,9.26211,-13.0106
7,4.06725,9.26211,-13.0106
8,9.25401,6.89455,-12.0875
9,6.88499,6.88499,-14.4187


In [7]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.01)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.01)

Processing point 1 of 49
Optimization has converged: ✓
Processing point 2 of 49
Optimization has converged: ✓
Processing point 3 of 49
Optimization has converged: ✓
Processing point 4 of 49
Optimization has converged: ✓
Processing point 5 of 49
Optimization has converged: ✓
Processing point 6 of 49
Optimization has converged: ✓
Processing point 7 of 49
Optimization has converged: ✓
Processing point 8 of 49
Optimization has converged: ✓
Processing point 9 of 49
Optimization has converged: ✓
Processing point 10 of 49
Optimization has converged: ✓
Processing point 11 of 49
Optimization has converged: ✓
Processing point 12 of 49
Optimization has converged: ✓
Processing point 13 of 49
Optimization has converged: ✓
Processing point 14 of 49
Optimization has converged: ✓
Processing point 15 of 49
Optimization has converged: ✓
Processing point 16 of 49
Optimization has converged: ✗
Processing point 17 of 49
Optimization has converged: ✓
Processing point 18 of 49
Optimization has converged: ✓
P

(49×8 DataFrame
 Row │ x1            x2        z           y1            y2           close  s ⋯
     │ Float64       Float64   Float64     Float64       Float64      Bool   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  9.26292       9.26292  -10.0763      -10.7742      -10.7742   false    ⋯
   2 │  6.89455       9.25401  -12.0875       10.7421       -7.67107  false
   3 │ -9.89628e-17   9.26948   -0.341423    -20.1316       -7.74024  false
   4 │ -6.89455       9.25401  -12.0875      -10.7421       -7.67107  false
   5 │ -9.26292       9.26292  -10.0763       10.7742      -10.7742   false    ⋯
   6 │ -4.06725       9.26211  -13.0106       -4.41908       1.47062  false
   7 │  4.06725       9.26211  -13.0106        4.41908       1.47062  false
   8 │  9.25401       6.89455  -12.0875       -7.67107      10.7421   false
  ⋮  │      ⋮           ⋮          ⋮            ⋮             ⋮         ⋮      ⋱
  43 │  9.26211       4.06725  -13.0106   

In [8]:
using GLMakie

In [9]:
plot_polyapprox_rotate(pol_lege, TR, df_lege, df_min_lege)

In [10]:
plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [11]:
# GLMakie.closeall()